# Max-P Regionalization for Multiple Components

**Authors:** **[Sergio Rey](https://github.com/sjsrey)**


The `max-p` problem involves the clustering of a set of geographic areas into the maximum number of homogeneous regions such that the value of a spatially extensive regional attribute is above a predefined threshold value. The spatially extensive attribute can be specified to ensure that each region contains sufficient population size, or a minimum number of enumeration units. The number of regions $p$ is endogenous to the problem and is useful for regionalization problems where the analyst does not require a fixed number of regions a-priori.

Originally formulated as a mixed-integer problem in [Duque, Anselin, Rey (2012)](https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1467-9787.2011.00743.x), `max-p` is an [NP-hard problem](https://en.wikipedia.org/wiki/NP-hardness) and exact solutions are only feasible for small problem sizes. As such, a number of heuristic solution approaches have been suggested. PySAL implements the heuristic approach described in
[Wei, Rey, and Knaap (2020)](https://www.tandfonline.com/doi/full/10.1080/13658816.2020.1759806).

One issue with the current version of maxp is when it is applied to a collection of areas that have multiple connected components *and* some of the components do not allow for feasible region building.

In this notebook we outline a number of alternatives to use when encountering infeasible components.

In [ ]:
%load_ext watermark
%watermark

In [ ]:
from spopt.region import MaxPHeuristic as MaxP
import matplotlib.pyplot as plt

import geopandas
import libpysal
import matplotlib
import numpy
import spopt
import warnings
from scipy.spatial import KDTree

plt.rcParams["figure.figsize"] = [12, 8]
warnings.filterwarnings("ignore")

RANDOM_SEED = 123456

%config InlineBackend.figure_format = "retina"
%watermark -w
%watermark -iv

In [ ]:
from shapely.geometry import Polygon, box

In [ ]:
n_cols = 5
n_rows = 10
b = 0
h = w = 10
component_0 = [box(l*w, b, l*w+w, b+h) for l in range(n_cols)] 
b = b + h*2
component_1 = [box(l*w, b + h * r, l * w + w, b +  h+ h * r) for r in range(n_rows) for l in range(n_cols) ] 
geometries = component_0 + component_1

In [ ]:
len(geometries)

In [ ]:
gdf = geopandas.GeoDataFrame(geometry=geometries, 
                             data = numpy.ones((n_cols*n_rows+n_cols,1), int),
                             columns=['var']
                            )
gdf.plot(edgecolor='black'), 

In [ ]:
gdf.head()

In [ ]:
w = libpysal.weights.Queen.from_dataframe(gdf)

In [ ]:
w.component_labels

So we have two components, one with 5 areas in the south and a larger northern component with 50 areas.

In [ ]:
import numpy 
numpy.unique(w.component_labels)
rng = numpy.random.default_rng(2021) # set seed for random numbers


We first attempt to solve where the threshold=5. The small southern component is feasible in this case so we should get a solution:

In [ ]:
model = MaxP(gdf, w, 'var', 'var', 5, 2, policy='keep')
model.solve()
gdf['region'] = model.labels_
gdf.explore(column='region', categorical=True)

Next we increase the threshold beyond the level of feasibility for this small component:

In [ ]:
try:
    model = MaxP(gdf, w, 'var', 'var', 6, 2, policy='keep')
    model.solve() 
except:
    print('No feasible solution!')


## Solution: `attach`
We can handle the small component by attaching each area in the infeasible component with its nearest neighbor area belonging to a feasible component. This is done with setting using the `attach` policy.

In [ ]:
model = MaxP(gdf, w, 'var', 'var', 6, 2, policy='attach')
model.solve() 
gdf['region'] = model.labels_
gdf.explore(column='region', categorical=True)

## Solution: `drop`

In [ ]:
from spopt.region.maxp import infeasible_components
gdf = geopandas.GeoDataFrame(geometry=geometries, 
                             data = numpy.ones((n_cols*n_rows+n_cols,1), int),
                             columns=['var']
                            )
gdf.plot(edgecolor='black'), 

In [ ]:
import numpy as np
model = MaxP(gdf, w, 'var', 'var', 6, 2, policy='drop')
model.solve() 
ifcs = infeasible_components(gdf, w, 'var', 6)
keep_ids = np.where(~np.isin(w.component_labels, ifcs))[0]
gdf['region'] = -1
gdf.region.iloc[keep_ids] = model.labels_
gdf.explore(column='region', categorical=True) # areas in region -1 are dropped

## No Feasible Components
We also handle the case when none of the components are feasible by rasing an exception.

In [ ]:
model = MaxP(gdf, w, 'var', 'var', 55, 2, policy='drop')
try:
    model.solve()
except Exception as e:
    print(e)